In [3]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats, integrate
import matplotlib.pyplot as plt
# import seaborn as sns
from numpy import sort
#import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import random
from sklearn import metrics, preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans

In [4]:
data_path = "F:\\Users\\ivann_000\\AnacondaProjects\\MLBootcamp4\\data"
X = pd.read_csv(data_path+"\\x_train.csv", sep=';', header=None)
y = pd.read_csv(data_path+"\\y_train.csv", sep=';', header=None)[0]

In [5]:
XX = X.copy()
corr_features = []
corr = XX.corr()
for i in corr:
    for j in corr:
        if ((abs(corr[i][j]) > 0.895) and (i!=j) and (i<j)):
            added = False
            for l, k in enumerate(corr_features):
                if not added:
                    if (i in k):
                        if j not in k:
                            k.append(j)
                        added = True
                    if (j in k):
                        if i not in k:
                            k.append(i)
                        added = True
            if not added:
                corr_features.append([i, j])
for i, c in enumerate(corr_features):
    for j, col in enumerate(c):
        if j == 0:
            XX['mean_'+str(i)]=XX[col]
        else:
            XX['mean_'+str(i)] += XX[col]
    XX['mean_'+str(i)] /= len(c)
#     for j, col in enumerate(c):
#         XX[str(col)+'_from_mean'] = XX['mean_'+str(i)] - XX[col]
features_to_drop = np.array([])
for f in corr_features:
    features_to_drop = np.append(features_to_drop, f)
print(features_to_drop)
XX = XX.drop(features_to_drop, axis=1)

[   0.    6.   26.   30.   51.   61.   70.  135.  138.  157.  183.  193.
  208.   23.   54.  120.  198.   55.  130.  143.  101.    1.    2.    7.
   46.   69.   94.  105.  117.  118.  167.  199.  200.  216.   81.  185.
   13.   89.  141.  174.  188.  215.   19.  186.   32.  201.  219.   41.
  184.  213.  218.   58.   59.   66.   76.   79.  221.   87.  102.  103.
  104.  107.  108.  128.  144.  150.  155.  168.  171.  190.    5.  163.
   36.  113.  172.   12.   13.   87.  125.  141.  174.   35.  102.  104.
  107.  128.  144.  170.  190.  195.   58.  155.   76.   17.   18.   27.
   59.   72.   79.  108.  114.  168.  171.  179.  221.  184.   22.   98.
  112.   28.   29.   57.  104.   43.   42.   32.   66.   90.   33.   77.
   84.  113.  127.  131.  139.  146.  148.  154.  162.  165.  172.  177.
   36.   92.  122.  136.  147.  212.  209.   34.   44.   45.   64.   97.
  116.  169.  187.  204.  210.   78.  220.   85.   99.  176.   35.   76.
   58.   41.  150.   47.   49.   62.  136.  147.  1

In [8]:
for comp in [100]:  
    pca = PCA(n_components=comp)
    XX_d = pd.DataFrame(pca.fit_transform(XX))
    model = XGBClassifier(n_estimators=1200,
                                   nthread=8,
                                   max_depth=15,
                                    learning_rate=0.01,
                                    colsample_bytree=0.5,
                                   seed=4242)
    print(comp, cross_val_score(model, XX_d, y, scoring='accuracy', cv=3).mean())

100 0.512188805124
